In [1]:
import pandas as pd
import string
import numpy as np
import glob
from keras.preprocessing.sequence import pad_sequences
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from tensorflow import keras

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [7]:
porter=PorterStemmer()
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y']
import pickle
with open('tokenizer-200.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [8]:
def clean_punct(a):        
    return a.lower().translate(str.maketrans('', '', string.punctuation))

def clean_stop(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
        if tk in stopwords:
            continue
        else:
            str1 += tk+" "
            
    return str1

def clean_stem(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
            str1 += porter.stem(tk)+" "
            
    return str1

In [11]:
from tensorflow import keras
model = keras.models.load_model('./Model-200-stem/')

count=0;

pozitive_total=0
negative_total=0
pozitive_corecte=0
negative_corecte=0
neutre_total=0
neutre_corecte=0
txtfiles = []
for file in glob.glob("Stiri_pozitive/*.txt"):
    txtfiles.append(file)
    

print("---Stiri Pozitive---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        #paragrafe=list(map(clean_stem, paragrafe))
        
        X_train = tokenizer.texts_to_sequences(paragrafe)
        X_train = pad_sequences(X_train, maxlen = 128)
        
        results=model.predict(X_train)
    
        
        sum_poz=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
            if np.count_nonzero(X_train[i]) > 3:
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_poz+=results[i]*len(str(paragrafe[i]).split())
        
        print(str(file)+"  "+str(sum_poz/nr_paragrafe))
        
        pozitive_total+=1
        if sum_poz/nr_paragrafe > 0.5:
            pozitive_corecte+=1

     
print("Total stiri pozitive : "+str(pozitive_total))
print("Corecte stiri pozitive : "+str(pozitive_corecte)) 
print("Acuratete Pozitive: "+str(pozitive_corecte/pozitive_total))
print("")    

txtfiles = []
for file in glob.glob("Stiri_negative/*.txt"):
    txtfiles.append(file)
print("---Stiri Negative---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        #paragrafe=list(map(clean_stem, paragrafe))
        
        X_train = tokenizer.texts_to_sequences(paragrafe)
        X_train = pad_sequences(X_train, maxlen = 128)
        
        results=model.predict(X_train)
        
        sum_neg=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_neg+=results[i]*len(str(paragrafe[i]).split())
                
        print(str(file)+"  "+str(sum_neg/nr_paragrafe))
        
        negative_total+=1
        if sum_neg/nr_paragrafe < 0.5:
            negative_corecte+=1
    
print("Total stiri negative : "+str(negative_total))
print("Corecte stiri negative : "+str(negative_corecte)) 
print("Acuratete negative: "+str(negative_corecte/negative_total))

print("")
print("Total stiri :" + str(pozitive_total+negative_total+neutre_total))
print("Total corecte :" + str(pozitive_corecte+negative_corecte+neutre_corecte))
print("Acuratete :" + str((pozitive_corecte+negative_corecte+neutre_corecte)/(pozitive_total+negative_total+neutre_total)))
    
    

---Stiri Pozitive---
Stiri_pozitive/34.txt  [0.70455205]
Stiri_pozitive/12.txt  [0.63913304]
Stiri_pozitive/11.txt  [0.42397076]
Stiri_pozitive/31.txt  [0.74565846]
Stiri_pozitive/3.txt  [0.695793]
Stiri_pozitive/17.txt  [0.6767607]
Stiri_pozitive/8.txt  [0.70980364]
Stiri_pozitive/10.txt  [0.43826136]
Stiri_pozitive/16.txt  [0.69200855]
Stiri_pozitive/23.txt  [0.6917656]
Stiri_pozitive/33.txt  [0.93476963]
Stiri_pozitive/30.txt  [0.60967046]
Stiri_pozitive/18.txt  [0.4758077]
Stiri_pozitive/7.txt  [0.63661784]
Stiri_pozitive/22.txt  [0.8056307]
Stiri_pozitive/6.txt  [0.642108]
Stiri_pozitive/29.txt  [0.8313422]
Stiri_pozitive/26.txt  [0.5971738]
Stiri_pozitive/27.txt  [0.7417906]
Stiri_pozitive/28.txt  [0.73767924]
Stiri_pozitive/0.txt  [0.7659139]
Stiri_pozitive/35.txt  [0.927807]
Stiri_pozitive/1.txt  [0.5521931]
Stiri_pozitive/21.txt  [0.4521298]
Stiri_pozitive/9.txt  [0.65424657]
Stiri_pozitive/24.txt  [0.6084637]
Stiri_pozitive/13.txt  [0.7540611]
Stiri_pozitive/14.txt  [0.617099